In [1]:
import torch

In [2]:
tensor0d = torch.tensor(1.0)
tensor1d = torch.tensor([1.0,2,3])
tensor2d = torch.tensor([[1.0,2,3],[4,5,6]])
tensor3d =  torch.tensor([[[1.0,2,3],[4,5,6]],[[7,8,9],[10,11,12]]])

In [3]:
print(tensor3d.dtype)

torch.float32


In [4]:
# tensor3d.ty

In [5]:
print(tensor3d)
print(tensor3d.shape)
# tensor mutiplication
x=tensor3d.reshape(3,2,2)
print(x.shape)
print(x)
y = torch.tensordot(tensor3d,x,1)
print(y.shape)

tensor([[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.]],

        [[ 7.,  8.,  9.],
         [10., 11., 12.]]])
torch.Size([2, 2, 3])
torch.Size([3, 2, 2])
tensor([[[ 1.,  2.],
         [ 3.,  4.]],

        [[ 5.,  6.],
         [ 7.,  8.]],

        [[ 9., 10.],
         [11., 12.]]])
torch.Size([2, 2, 2, 2])


In [6]:
# 2d tensor multiplicaiton
# can be done as shown above (contracting over 1 dimension)
# or as shown below
print(tensor2d)
print(tensor2d.T)
print(torch.matmul(tensor2d,tensor2d.T))
print(tensor2d @ tensor2d.T)

tensor([[1., 2., 3.],
        [4., 5., 6.]])
tensor([[1., 4.],
        [2., 5.],
        [3., 6.]])
tensor([[14., 32.],
        [32., 77.]])
tensor([[14., 32.],
        [32., 77.]])


In [7]:
from torch.nn.functional import binary_cross_entropy as bce
y = torch.tensor(1.0)
x1 = torch.tensor(1.1)
w1 = torch.tensor(2.2)
b = torch.tensor(0.0)
z = x1 * w1 + b
a = torch.sigmoid(z)
loss = bce(a,y)
print(loss)

tensor(0.0852)


In [8]:
# A.4 Computing Gradients via autograd
from torch.autograd import grad

y = torch.tensor(1.0)
x1 = torch.tensor(1.1)
w1 = torch.tensor(2.2, requires_grad = True)
b = torch.tensor(0.0, requires_grad = True)

z = x1 * w1 + b
a = torch.sigmoid (z)
loss = bce(a,y)

grad_L_w1 = grad(loss, w1, retain_graph = True)
grad_L_b = grad(loss, b, retain_graph = True)
print(grad_L_w1)
print(grad_L_b)

loss.backward()
print(w1.grad)
print(b.grad)

(tensor(-0.0898),)
(tensor(-0.0817),)
tensor(-0.0898)
tensor(-0.0817)


In [9]:
class NeuralNetwork (torch.nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super().__init__()

        self.layers = torch.nn.Sequential(
            # first hiddel layer
            torch.nn.Linear(num_inputs, 30),
            torch.nn.ReLU(),

            #2nd hidned layeyr
            torch.nn.Linear(30,20),
            torch.nn.ReLU(),
            # outpt layer
            torch.nn.Linear(20, num_outputs)
        )
    def forward(self, x):
        logits = self.layers(x)
        return logits


In [10]:
model = NeuralNetwork(50, 3)
print(model)

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)


In [11]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print('Number of trainable model paramerers:', num_params)

Number of trainable model paramerers: 2213


In [12]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [13]:
model.layers[0].bias.shape

torch.Size([30])

In [ ]:
torch.manual_seed(123)
model = NeuralNetwork(50,3)
print(model.layers[0].weight)

Parameter containing:
tensor([[-0.0577,  0.0047, -0.0702,  ...,  0.0222,  0.1260,  0.0865],
        [ 0.0502,  0.0307,  0.0333,  ...,  0.0951,  0.1134, -0.0297],
        [ 0.1077, -0.1108,  0.0122,  ...,  0.0108, -0.1049, -0.1063],
        ...,
        [-0.0787,  0.1259,  0.0803,  ...,  0.1218,  0.1303, -0.1351],
        [ 0.1359,  0.0175, -0.0673,  ...,  0.0674,  0.0676,  0.1058],
        [ 0.0790,  0.1343, -0.0293,  ...,  0.0344, -0.0971, -0.0509]],
       requires_grad=True)


In [15]:
X = torch.rand((1,50))
out = model (X)
print(out)

tensor([[-0.1670,  0.1001, -0.1219]], grad_fn=<AddmmBackward0>)


In [ ]:
with torch.no_grad():
    out = torch.softmax (model (X), dim = 1)
print(out)

tensor([[0.2983, 0.3896, 0.3121]])


In [17]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0,0,0,1,1])

X_test = torch.tensor([[-0.8, 2.8],[2.6, -106]])

y_test = torch.tensor([0,1])


In [18]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__ (self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):
        one_x = self.features[index]
        one_y = self.labels[index]
        return one_x, one_y
    def __len__(self):
        return self.labels.shape[0]
train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [19]:
print(len(train_ds))

5


In [20]:
from torch.utils.data import DataLoader

torch.manual_seed(123)

train_loader = DataLoader (
    dataset = train_ds,
    batch_size = 2,
    shuffle = True,
    num_workers = 0,
    drop_last = True
)

test_loader = DataLoader(
    dataset = test_ds,
    batch_size = 2,
    shuffle = False,
    num_workers = 0
)

In [ ]:
for idx, (x,y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x ,y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[-1.2000,  3.1000],
        [-0.5000,  2.6000]]) tensor([0, 0])


In [22]:
import torch.nn.functional as F

torch.manual_seed(123)

model = NeuralNetwork(num_inputs=2, num_outputs=2)
optimizer = torch.optim.SGD (model.parameters(), lr = 0.5)

num_epochs = 3
for epoch in range (num_epochs):

    model.train()
    for batch_idx, (features,labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
        f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
        f" | Train Loss: {loss:.2f}")
    model.eval()

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.75
Epoch: 001/003 | Batch 001/002 | Train Loss: 0.65
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.44
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.13
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.03
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00


In [23]:
model.eval()
with torch.no_grad():
    outputs = model(X_train)
print(outputs)

tensor([[ 2.8569, -4.1618],
        [ 2.5382, -3.7548],
        [ 2.0944, -3.1820],
        [-1.4814,  1.4816],
        [-1.7176,  1.7342]])


In [ ]:
torch.set_printoptions (sci_mode = False)
probas = torch.softmax(outputs, dim =1)
print (probas)

tensor([[    0.9991,     0.0009],
        [    0.9982,     0.0018],
        [    0.9949,     0.0051],
        [    0.0491,     0.9509],
        [    0.0307,     0.9693]])


In [25]:
predictions  = torch.argmax(probas, dim=1)
print(predictions)
print(predictions == y_train)
print(torch.sum(predictions == y_train))

tensor([0, 0, 0, 1, 1])
tensor([True, True, True, True, True])
tensor(5)


In [26]:
def compute_accuracy (model, dataloader):

    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate (dataloader):

        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim = 1)
        compare = labels ==predictions
        correct += torch.sum(compare)
        total_examples += len(compare)

    return (correct / total_examples).item()

In [ ]:
print(compute_accuracy(model, train_loader))

1.0


In [28]:
print(compute_accuracy(model, test_loader))

1.0


In [29]:
torch.save(model.state_dict(), 'model.pth')

In [ ]:
model = NeuralNetwork(2,2)
model.load_state_dict(torch.load('model.pth', weights_only=1))

<All keys matched successfully>

In [31]:
print (torch.cuda.is_available())

True


In [32]:
tensor_1 = torch.tensor([1.,2.,3.])
tensor_2 = torch.tensor([4.,5.,6.])
print(tensor_1 + tensor_2)

tensor([5., 7., 9.])


In [33]:
tensor_1 = tensor_1.to('cuda')
tensor_2 = tensor_2.to('cuda')
print(tensor_1 + tensor_2)

tensor([5., 7., 9.], device='cuda:0')


In [34]:
torch.manual_seed(123)
model = NeuralNetwork(num_inputs = 2, num_outputs = 2)
device = torch.device('cuda')
model = model.to(device)

optimiser = torch.optim.SGD(model.parameters(), lr = 0.5)

num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        features, labels = features.to(device), labels.to(device)
        